<a href="https://colab.research.google.com/github/PiyumaliSandunika/e18-4yp-Multimodal-Emotion-Prediction-Using-Reinforcement-Learning/blob/main/bert3_last_stable_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split


In [ ]:
# Load dataset
df = pd.read_csv("e_data.csv")

In [ ]:
df['sentiment'].value_counts()

sentiment
angry      150
joy        150
neutral    150
sad        150
Name: count, dtype: int64

In [ ]:
df_org = df.sample(frac=1.0, random_state=42)
df_org = df_org.drop('tweet_id',axis =1 )
# rename 'hate'as 'anger' in sentiments
df_org['sentiment'] = df_org['sentiment'].replace('hate','anger')
# rename love,fun as joy
df_org['sentiment'] = df_org['sentiment'].replace(['love','fun'],'happiness')
# rename worry,boredom as sadness
df_org['sentiment'] = df_org['sentiment'].replace('worry','sadness')
df_org = df_org[df_org['sentiment'].apply(lambda x: x not in ['worry','love','fun','surprise','relief','hate','empty','enthusiasm','boredom'])]
print(df_org['sentiment'].value_counts())

sentiment
sad        150
joy        150
neutral    150
angry      150
Name: count, dtype: int64


In [ ]:
min_count = df_org['sentiment'].value_counts().min()
df_org = df_org.groupby('sentiment').apply(lambda x: x.sample(min_count)).reset_index(drop=True)
print(df_org['sentiment'].value_counts())


sentiment
angry      150
joy        150
neutral    150
sad        150
Name: count, dtype: int64


In [ ]:
df = df_org

In [ ]:
# Extract sentences and labels
texts = df["content"].tolist()
labels = df["sentiment"].tolist()

In [ ]:
# Convert emotion labels to numerical values
label_map = {"happiness": 0, "sadness": 1, "anger": 2, "neutral": 3}
labels = [label_map[label] for label in labels]


In [ ]:
from sklearn.model_selection import train_test_split
# Split data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)
# split dataset into train validation and test



In [1]:
# Load pre-trained DistilBERT model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


NameError: name 'DistilBertForSequenceClassification' is not defined

In [ ]:
# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


In [ ]:
# Create PyTorch datasets
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)



In [ ]:
train_dataset = EmotionDataset(train_encodings, train_labels)
val_dataset = EmotionDataset(val_encodings, val_labels)


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Define hyperparameter search space
param_grid = {
    'learning_rate': [1e-5, 2e-5, 3e-5],
    'batch_size': [16, 32, 64],
    'num_train_epochs': [3, 4, 5],
    'weight_decay': [0.01, 0.001, 0.0001],
    'dropout_rate': [0.1, 0.2, 0.3]
}

# # Define model and tokenizer
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')



In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

In [ ]:
from sklearn.metrics import accuracy_score

# Define a function to compute accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy}

In [ ]:
# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# # Define hyperparameter search space
# param_grid = {
#     'learning_rate': [1e-5, 2e-5, 3e-5],
#     'batch_size': [16, 32, 64],
#     'num_train_epochs': [3, 4, 5],
#     'weight_decay': [0.01, 0.001, 0.0001],
#     'dropout_rate': [0.1, 0.2, 0.3]
# }

# # Define model and tokenizer
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# # Define Trainer and TrainingArguments (not shown here)

# # Define RandomizedSearchCV
# random_search = RandomizedSearchCV(estimator=Trainer(model=model, args=training_args),
#                                    param_distributions=param_grid,
#                                    n_iter=10,
#                                    scoring='accuracy',
#                                    cv=3,
#                                    verbose=2,
#                                    random_state=42,
#                                    n_jobs=-1)

# # Perform random search
# random_search.fit(train_dataset)

# # Get best hyperparameters
# best_hyperparameters = random_search.best_params_

# print("Best Hyperparameters:", best_hyperparameters)


In [ ]:
# Train the model
h = trainer.train()


Step,Training Loss


In [ ]:
evaluation_results = trainer.evaluate()

# Print accuracy
print("Accuracy:", evaluation_results['eval_accuracy'])

Accuracy: 0.95


In [ ]:
# Save the trained model
trainer.save_model("emotion_classification_model")

In [ ]:
import pickle

# Save the trained model
model_path = "emotion_text_classification_model.pkl"
with open(model_path, 'wb') as f:
    pickle.dump(model, f)


** Model Loading**

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch

# Load the saved model
model_path = "emotion_classification_model"
loaded_model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [ ]:
model_path2 = "emotion_classification_model_lite"
loaded_model2 = DistilBertForSequenceClassification.from_pretrained(model_path)
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [ ]:
def predict_emotion_with_probabilities(sentence, model, tokenizer):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt")

    # Perform inference
    outputs = model(**inputs)
    logits = outputs.logits

    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # Convert probabilities tensor to list
    probabilities_list = probabilities.squeeze().tolist()

    # Map predicted label to emotion
    emotion_labels = ["happiness", "sadness", "anger", "neutral"]
    predicted_emotion = emotion_labels[torch.argmax(logits, dim=1).item()]

    return probabilities_list, predicted_emotion


In [ ]:
def ensemble_predictions(sentence, model_weights=None, class_weights=None):
    # Get predictions from the first model
    probabilities1, _ = predict_emotion_with_probabilities(sentence, loaded_model, tokenizer)

    # Get predictions from the second model
    probabilities2, _ = predict_emotion_with_probabilities(sentence, loaded_model2, tokenizer)

    # Apply model-wise weights if provided
    if model_weights:
        probabilities1 = [p1 * w for p1, w in zip(probabilities1, model_weights[0])]
        probabilities2 = [p2 * w for p2, w in zip(probabilities2, model_weights[1])]

    # Apply class-wise weights if provided
    if class_weights:
        probabilities1 = [p1 * w for p1, w in zip(probabilities1, class_weights)]
        probabilities2 = [p2 * w for p2, w in zip(probabilities2, class_weights)]

    # Calculate ensemble probabilities by averaging
    ensemble_probabilities = [(p1 + p2) / 2 for p1, p2 in zip(probabilities1, probabilities2)]

    # Map ensemble probabilities to emotion labels
    emotion_labels = ["happiness", "sadness", "anger", "neutral"]
    predicted_emotion = emotion_labels[np.argmax(ensemble_probabilities)]

    return probabilities1, probabilities2, ensemble_probabilities, predicted_emotion

# Example usage with model and class weights
sentence = "This is a test sentence."
model_weights = [[1.0, 1.0, 1.0, 1.0], [0.8, 0.8, 0.9, 1.2]]  # Example weights for each model
class_weights = [1.0, 1.0, 0.8, 1.2]  # Example weights for each class
probabilities1, probabilities2, ensemble_probabilities, predicted_emotion = ensemble_predictions(sentence, model_weights, class_weights)
print("Probabilities (Model 1):", probabilities1)
print("Probabilities (Model 2):", probabilities2)
print("Ensemble Probabilities:", ensemble_probabilities)
print("Ensemble Predicted Emotion:", predicted_emotion)


Probabilities (Model 1): [0.0003059331502299756, 0.0008683084743097425, 0.0019081057980656624, 1.1957287073135376]
Probabilities (Model 2): [0.0002447465201839805, 0.000694646779447794, 0.0017172952182590963, 1.434874448776245]
Ensemble Probabilities: [0.000275339835206978, 0.0007814776268787682, 0.0018127005081623794, 1.3153015780448913]
Ensemble Predicted Emotion: neutral


In [ ]:

# Example sentences for prediction
sentences = [
    "I love going to the park, it always makes me happy.",
    "The loss of my pet dog left me feeling devastated.",
    "I'm afraid of the dark, it gives me chills.",
    "The weather today is quite pleasant."
]

# Perform prediction for each sentence
for sentence in sentences:
    probabilities_list, predicted_emotion = predict_emotion(sentence)
    print(f"Sentence: '{sentence}'")
    print(f"Predicted Emotion: {probabilities_list}\n")
    print(f"Predicted Emotion: {predicted_emotion}\n")

Sentence: 'I love going to the park, it always makes me happy.'
Predicted Emotion: [0.5162611603736877, 0.18221308290958405, 0.15279607474803925, 0.14872965216636658]

Predicted Emotion: joy

Sentence: 'The loss of my pet dog left me feeling devastated.'
Predicted Emotion: [0.2505452036857605, 0.37606674432754517, 0.2137974351644516, 0.15959052741527557]

Predicted Emotion: sad

Sentence: 'I'm afraid of the dark, it gives me chills.'
Predicted Emotion: [0.2281908094882965, 0.3160349130630493, 0.25184106826782227, 0.20393317937850952]

Predicted Emotion: sad

Sentence: 'The weather today is quite pleasant.'
Predicted Emotion: [0.40060093998908997, 0.1721194088459015, 0.1856856793165207, 0.24159397184848785]

Predicted Emotion: joy



In [ ]:

# Example sentences for prediction
sentences = [
    "I am a broken-hearted",
    "I am emotionaly damaged right now",
    "Do what makes feel you better",
    "I feel nervous",
    "I am a student",
    "You're stupid and it makes me anger",
    "Go to hell",
    "You are the worst,I do not want to see you",
    "I am angry right now",
    "You are my sunshine",
    "Such a wonderful performance",
    "I am lonely"
]

# Perform prediction for each sentence
for sentence in sentences:
    predicted_emotion = predict_emotion(sentence)
    print(f"Sentence: '{sentence}'")
    print(f"Predicted Emotion: {predicted_emotion}\n")

Sentence: 'I am a broken-hearted'
Predicted Emotion: ([0.00020950073667336255, 0.9991468191146851, 0.00019554900063667446, 0.00044807567610405385], 'sadness')

Sentence: 'I am emotionaly damaged right now'
Predicted Emotion: ([0.00024238182231783867, 0.9989521503448486, 0.000303982145851478, 0.0005014440976083279], 'sadness')

Sentence: 'Do what makes feel you better'
Predicted Emotion: ([0.0008129614870995283, 0.0006066274363547564, 0.00010575642954790965, 0.998474657535553], 'neutral')

Sentence: 'I feel nervous'
Predicted Emotion: ([0.0005710084806196392, 0.9955019354820251, 0.0003445712791290134, 0.003582351142540574], 'sadness')

Sentence: 'I am a student'
Predicted Emotion: ([4.979088043910451e-05, 0.00010799858864629641, 9.025849431054667e-05, 0.9997519850730896], 'neutral')

Sentence: 'You're stupid and it makes me anger'
Predicted Emotion: ([0.00018839072436094284, 0.0004287665069568902, 0.9992074370384216, 0.0001754151890054345], 'anger')

Sentence: 'Go to hell'
Predicted Emo

In [ ]:

# Example sentences for prediction
sentences = [
    "I am a broken-hearted",
    "I am emotionaly damaged right now",
    "Do what makes feel you better",
    "I feel nervous",
    "I am a student",
    "You're stupid and it makes me anger",
    "Go to hell",
    "You are the worst,I do not want to see you",
    "I am angry right now",
    "You are my sunshine",
    "Such a wonderful performance",
    "I am lonely"
]

# Perform prediction for each sentence
for sentence in sentences:
    predicted_emotion = predict_emotion(sentence)
    print(f"Sentence: '{sentence}'")
    print(f"Predicted Emotion: {predicted_emotion}\n")

Sentence: 'I am a broken-hearted'
Predicted Emotion: ([0.22256985306739807, 0.2726081311702728, 0.30142638087272644, 0.20339569449424744], 'angry')

Sentence: 'I am emotionaly damaged right now'
Predicted Emotion: ([0.22510485351085663, 0.2804677188396454, 0.28153783082962036, 0.2128896266222], 'angry')

Sentence: 'Do what makes feel you better'
Predicted Emotion: ([0.3086329996585846, 0.20802205801010132, 0.17631801962852478, 0.3070269823074341], 'joy')

Sentence: 'I feel nervous'
Predicted Emotion: ([0.21537865698337555, 0.3466716706752777, 0.24246057868003845, 0.1954890787601471], 'sad')

Sentence: 'I am a student'
Predicted Emotion: ([0.29096055030822754, 0.22324411571025848, 0.26276153326034546, 0.2230338305234909], 'joy')

Sentence: 'You're stupid and it makes me anger'
Predicted Emotion: ([0.21656301617622375, 0.2647278904914856, 0.34241899847984314, 0.17629007995128632], 'angry')

Sentence: 'Go to hell'
Predicted Emotion: ([0.23550142347812653, 0.2363341897726059, 0.25273290276

In [ ]:
# import pickle

# # Load the model from the pickle file
# with open("test_based_recog_new.pkl", "rb") as f:
#     loaded_model2 = pickle.load(f)


In [ ]:
# max_len =50

# from tensorflow.keras.preprocessing.sequence import pad_sequences
# def get_sequences(tokenizer, tweets):
#     sequences = tokenizer.texts_to_sequences(tweets)
#     padded = pad_sequences(sequences,truncating= 'post',padding='post', maxlen =max_len)
#     return padded

In [ ]:
# # Example usage
# def predict2(sentence):
#     import numpy as np
#     from tensorflow.keras.preprocessing.text import Tokenizer
#     from tensorflow.keras.preprocessing.sequence import pad_sequences

#     tokenizer  = Tokenizer(num_words = 1000, oov_token = '<UNK>' )
#     seq_test = get_sequences(tokenizer, sentence)
#     print(seq_text)
#     # p = loaded_model2.predict(seq_test)
#     # p = np.argmax(p).astype('uint8')
#     # print(p)
#     # print('Prediction : ',get_emotion_name(p))